In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [ ]:
# from denoisplit.core.tiff_reader import load_tiff
# fname = '/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk44/pred_disentangle_2402_D16-M23-S0-L0_31.tif'
# data = load_tiff(fname)

In [ ]:
# there are two environments(debug and prod). From where you want to fetch the code and data? 
DEBUG=False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
# from denoisplit.core.tiff_reader import load_tiff
# import matplotlib.pyplot as plt
# # data = load_tiff('/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk44/pred_disentangle_2402_D16-M23-S0-L0_88.tif')
# # plt.imshow(data[0,...,0])

In [ ]:
ckpt_dir = "/home/ashesh.ashesh/training/disentangle/2403/D7-M23-S0-L0/32"
# 211/D3-M3-S0-L0/0
# 2210/D3-M3-S0-L0/128
# 2210/D3-M3-S0-L0/129

In [ ]:
# !ls /home/ubuntu/ashesh/training/disentangle/2209/D3-M9-S0-L0/1

In [ ]:
dtype = int(ckpt_dir.strip('/').split('/')[-2].split('-')[0][1:])

In [ ]:
dtype

In [ ]:
import json
if DEBUG:
    if dtype == DataType.CustomSinosoid:
        data_dir = f'{DATA_ROOT}/sinosoid/'
    elif dtype == DataType.OptiMEM100_014:
        data_dir = f'{DATA_ROOT}/microscopy/'
else:
    if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
        data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
    elif dtype == DataType.OptiMEM100_014:
        data_dir = f'{DATA_ROOT}/microscopy/'
    elif dtype == DataType.Prevedel_EMBL:
        data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
    elif dtype == DataType.AllenCellMito:
        data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
    elif dtype == DataType.SeparateTiffData:
        data_dir = f'{DATA_ROOT}/ventura_gigascience'
    elif dtype == DataType.SemiSupBloodVesselsEMBL:
        data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
    elif dtype == DataType.Pavia2VanillaSplitting:
        data_dir = f'{DATA_ROOT}/pavia2'
    elif dtype == DataType.ExpansionMicroscopyMitoTub:
        data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
    elif dtype == DataType.ShroffMitoEr:
        data_dir = f'{DATA_ROOT}/shrofflab/'
    elif dtype == DataType.HTIba1Ki67:
        data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
    elif dtype == DataType.BioSR_MRC:
        data_dir = f'{DATA_ROOT}/BioSR/'
        
#     2720*2720: microscopy dataset.

image_size_for_grid_centers = 32
mmse_count = 2
custom_image_size = 128
denoise_channel = None
save_output = False
save_output_dir = f'/group/jug/ashesh/data/denoiser_output/{os.path.basename(data_dir)}'

batch_size = 8
num_workers = 1
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'

if save_output:
    assert eval_datasplit_type == DataSplitType.All
    assert save_output_dir is not None
    assert os.path.exists(save_output_dir), f"{save_output_dir} does not exist"
    with open(f'{save_output_dir}/config.json', 'w') as f:
        json.dump({'ckpt_dir': ckpt_dir, 
                   'data_dir': data_dir, 
                   'image_size_for_grid_centers': image_size_for_grid_centers, 
                   'mmse_count': mmse_count, 
                   'custom_image_size': custom_image_size, 
                   'denoise_channel': denoise_channel, 
                   'use_deterministic_grid': use_deterministic_grid, 
                   'threshold': threshold, 
                  'eval_datasplit_type': eval_datasplit_type, 
                  'val_repeat_factor': val_repeat_factor}, f)

In [ ]:
%run ./nb_core/config_loader.ipynb

In [ ]:
# config.model.decoder

In [ ]:
from denoisplit.core.sampler_type import SamplerType
from denoisplit.core.loss_type import LossType
from denoisplit.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from denoisplit.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if denoise_channel is not None:
        config.model.denoise_channel = denoise_channel
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.Iba1Ki64
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0

In [ ]:
if denoise_channel is None:
    denoise_channel = config.model.denoise_channel 
    print(f"denoise_channel: {denoise_channel}")


In [ ]:
%run ./nb_core/disentangle_setup.ipynb

In [ ]:
len(train_dset)

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
# if config.model.model_type not in [ModelType.UNet, ModelType.BraveNet]:
#     with torch.no_grad():
#         inp, tar = val_dset[0][:2]
#         out, td_data = model(torch.Tensor(inp[None]).cuda())
#         print(td_data['z'][-1].shape)
#         print(out.shape)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = max(len(inp_tmp),3)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(len(inp_tmp)):
    ax[0,i].imshow(inp_tmp[i])

ax[1,0].imshow(tar_tmp[0]+tar_tmp[1])
ax[1,1].imshow(tar_tmp[0])
ax[1,2].imshow(tar_tmp[1])


In [ ]:
from denoisplit.analysis.stitch_prediction import stitch_predictions
from denoisplit.analysis.mmse_prediction import get_dset_predictions
# from denoisplit.analysis.stitch_prediction import get_predictions as get_dset_predictions

pred_tiled, rec_loss, logvar, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
assert patch_psnr_tuple[1] is None
print('Patch wise PSNR, as computed during training', np.round(patch_psnr_tuple[0].item(),2))

In [ ]:
pred_tiled.shape

In [ ]:
idx_list = np.where(logvar.squeeze() < -6)[0]
if len(idx_list) > 0:
    plt.imshow(val_dset[idx_list[0]][1][1])

In [ ]:
len(val_dset)

In [ ]:
plt.imshow((val_dset._data[0,...,1] + val_dset._data[0,...,0])/2)

In [ ]:
import seaborn as sns
sns.histplot(logvar[::50].squeeze().reshape(-1,))

In [ ]:
print(np.quantile(rec_loss, [0,0.01,0.5, 0.9,0.99,0.999,1]).round(2))

In [ ]:
pred_tiled.shape

In [ ]:
len(val_dset)

In [ ]:
if pred_tiled.shape[-1] != val_dset.get_img_sz():
    pad = (val_dset.get_img_sz() - pred_tiled.shape[-1] )//2
    pred_tiled = np.pad(pred_tiled, ((0,0),(0,0),(pad,pad),(pad,pad)))

pred = stitch_predictions(pred_tiled,val_dset, smoothening_pixelcount=0)

In [ ]:
if pred.shape[-1] != pred_tiled.shape[1]:
    assert pred.shape[-1] == 1 + pred_tiled.shape[1]
    assert pred[...,-1].std() == 0
    pred = pred[...,:-1].copy()
    # pred_std = pred_std[...,:-1].copy()
    if logvar is not None:
        logvar = logvar[...,:-1].copy()

In [ ]:
pred.shape

In [ ]:
pred[np.isnan(pred)] = 0

In [ ]:
def print_ignored_pixels():
    ignored_pixels = 1
    while(pred[:10,-ignored_pixels:,-ignored_pixels:,].std() ==0):
        ignored_pixels+=1
    ignored_pixels-=1
    print(f'In {pred.shape}, last {ignored_pixels} many rows and columns are all zero.')
    return ignored_pixels

actual_ignored_pixels = print_ignored_pixels()

## Ignore the pixels which are present in the last few rows and columns. 
1. They don't come in the batches. So, in prediction, they are simply zeros. So they are being are ignored right now. 
2. For the border pixels which are on the top and the left, overlapping yields worse performance. This is becuase, there is nothing to overlap on one side. So, they are essentially zero padded. This makes the performance worse. 

In [ ]:
actual_ignored_pixels

In [ ]:
if config.data.data_type in [DataType.OptiMEM100_014,
                                                      DataType.SemiSupBloodVesselsEMBL, 
                                                      DataType.Pavia2VanillaSplitting,
                                                      DataType.ExpansionMicroscopyMitoTub,
                                                      DataType.ShroffMitoEr,
                                                      DataType.HTIba1Ki67]:
    ignored_last_pixels = 32 
elif config.data.data_type == DataType.BioSR_MRC:
    ignored_last_pixels = 44
    # assert val_dset.get_img_sz() == 64
else:
    ignored_last_pixels = 0

ignore_first_pixels = 0

assert actual_ignored_pixels <= ignored_last_pixels, f'Set ignored_last_pixels={actual_ignored_pixels}'
print(ignored_last_pixels)

In [ ]:
if model.denoise_channel == 'Ch1':
    tar = val_dset._data[...,:1]
elif model.denoise_channel == 'Ch2':
    tar = val_dset._data[...,1:]
elif model.denoise_channel == 'input':
    tar = np.mean(val_dset._data, axis=-1, keepdims=True)
    

def ignore_pixels(arr):
    if ignore_first_pixels:
        arr = arr[:,ignore_first_pixels:,ignore_first_pixels:]
    if ignored_last_pixels:
        arr = arr[:,:-ignored_last_pixels,:-ignored_last_pixels]
    return arr

pred = ignore_pixels(pred)
tar = ignore_pixels(tar)


In [ ]:
from skimage.metrics import structural_similarity

def _avg_psnr(target, prediction, psnr_fn):
    output = np.mean([psnr_fn(target[i:i + 1], prediction[i:i + 1]).item() for i in range(len(prediction))])
    return round(output, 2)


def avg_range_inv_psnr(target, prediction):
    return _avg_psnr(target, prediction, RangeInvariantPsnr)


def avg_psnr(target, prediction):
    return _avg_psnr(target, prediction, PSNR)


def compute_masked_psnr(mask, tar1, tar2, pred1, pred2):
    mask = mask.astype(bool)
    mask = mask[..., 0]
    tmp_tar1 = tar1[mask].reshape((len(tar1), -1, 1))
    tmp_pred1 = pred1[mask].reshape((len(tar1), -1, 1))
    tmp_tar2 = tar2[mask].reshape((len(tar2), -1, 1))
    tmp_pred2 = pred2[mask].reshape((len(tar2), -1, 1))
    psnr1 = avg_range_inv_psnr(tmp_tar1, tmp_pred1)
    psnr2 = avg_range_inv_psnr(tmp_tar2, tmp_pred2)
    return psnr1, psnr2

def avg_ssim(target, prediction):
    ssim = [structural_similarity(target[i],prediction[i], data_range=(target[i].max() - target[i].min())) for i in range(len(target))]
    return np.mean(ssim),np.std(ssim)

In [ ]:
sep_mean, sep_std = model.data_mean, model.data_std
if isinstance(sep_mean, dict):
    sep_mean = sep_mean['target']
    sep_std = sep_std['target']

if isinstance(sep_mean, int):
    pass
else:
    sep_mean = sep_mean.squeeze()[None,None,None]
    sep_std = sep_std.squeeze()[None,None,None]
    sep_mean = sep_mean.cpu().numpy() 
    sep_std = sep_std.cpu().numpy()

tar_normalized = (tar - sep_mean)/ sep_std

In [ ]:
q_vals = [0.01, 0.1,0.5,0.9,0.95, 0.99,1]
print(np.quantile(tar_normalized[...,0], q_vals).round(2))

In [ ]:
print(np.quantile(tar[...,0], q_vals))


In [ ]:
_,ax = plt.subplots(figsize=(6,6))
# sns.histplot(tar[:,...,0].reshape(-1,), color='g', label='Nuc')
# sns.histplot(tar[:,...,1].reshape(-1,), color='r', label='Tub')

sns.histplot(tar[:,::10,::10,0].reshape(-1,), color='g', kde=True)

In [ ]:
# from denoisplit.data_loader.schroff_rawdata_loader import mito_channel_fnames
# from denoisplit.core.tiff_reader import load_tiff
# import seaborn as sns

# fpaths = [os.path.join(datapath, x) for x in mito_channel_fnames()]
# fpath = fpaths[0]
# print(fpath)
# img = load_tiff(fpaths[0])
# temp = img.copy()
# sns.histplot(temp[:,:,::10,::10].reshape(-1,))
# plt.hist(temp[:,:,::10,::10].reshape(-1,),bins=100)

In [ ]:
import matplotlib.patches as patches
import matplotlib
from denoisplit.analysis.plot_error_utils import plot_error

_,ax = plt.subplots(figsize=(12,8),ncols=3,nrows=2)
idx = np.random.randint(len(pred))
print(idx)
ax[0,0].imshow(tar_normalized[idx,...,0], cmap='magma')
ax[0,1].imshow(pred[idx,:,:,0], cmap='magma')
plot_error(tar_normalized[idx,...,0], 
           pred[idx,:,:,0], 
           cmap = matplotlib.cm.coolwarm, 
           ax = ax[0,2], max_val = None)

cropsz = 512
h_s = np.random.randint(0, tar_normalized.shape[1] - cropsz)
h_e = h_s + cropsz
w_s = np.random.randint(0, tar_normalized.shape[2] - cropsz)
w_e = w_s + cropsz

ax[1,0].imshow(tar_normalized[idx,h_s:h_e,w_s:w_e,0], cmap='magma')
ax[1,1].imshow(pred[idx,h_s:h_e,w_s:w_e,0], cmap='magma')
plot_error(tar_normalized[idx,h_s:h_e,w_s:w_e,0], 
           pred[idx,h_s:h_e,w_s:w_e,0], 
           cmap = matplotlib.cm.coolwarm, 
           ax = ax[1,2], max_val = None)



clean_ax(ax[0,3:])

# Add rectangle to the region
rect = patches.Rectangle((w_s, h_s), w_e-w_s, h_e-h_s, linewidth=1, edgecolor='r', facecolor='none')
ax[0,2].add_patch(rect)
# plt.colorbar()
pred.shape

In [ ]:
# ch1_pred_unnorm = pred[...,0]*sep_std[...,0].cpu().numpy() + sep_mean[...,0].cpu().numpy()
# ch2_pred_unnorm = pred[...,1]*sep_std[...,1].cpu().numpy() + sep_mean[...,1].cpu().numpy()
pred_unnorm = pred*sep_std + sep_mean

In [ ]:
np.sqrt(((pred[...,:1] - tar_normalized)**2).reshape(len(pred),-1).mean(axis=1)).mean()

In [ ]:
print(sep_mean.squeeze(), sep_std.squeeze(), pred.shape )

In [ ]:
assert pred.shape == tar_normalized.shape, f"pred.shape: {pred.shape}, tar_normalized.shape: {tar_normalized.shape}"
rmse =np.sqrt(((pred - tar_normalized)**2).reshape(len(pred),-1).mean(axis=1))
rmse = np.round(rmse,3)
psnr = avg_psnr(tar_normalized[...,0].copy(), pred[...,0].copy()) 
rinv_psnr = avg_range_inv_psnr(tar_normalized[...,0].copy(), pred[...,0].copy())
ssim_mean, ssim_std = avg_ssim(tar[...,0], pred_unnorm[...,0])

In [ ]:
print(f'{DataSplitType.name(eval_datasplit_type)}_P{custom_image_size}_G{image_size_for_grid_centers}_M{mmse_count}_Sk{ignored_last_pixels}')
print('Rec Loss',model.denoise_channel, np.round(rec_loss.mean(),3) )
print('RMSE', model.denoise_channel, np.mean(rmse).round(3))
print('PSNR',model.denoise_channel, psnr)
print('RangeInvPSNR',model.denoise_channel, rinv_psnr)
print('SSIM',model.denoise_channel, round(ssim_mean,3),'±',round(ssim_std,4))
print()

In [ ]:
from denoisplit.scripts.evaluate import * 
highres_data = None

if config.model.model_type == ModelType.DenoiserSplitter or config.data.data_type == DataType.SeparateTiffData:
    from denoisplit.scripts.evaluate import get_highres_data_ventura
    highres_data = get_highres_data_ventura(data_dir, config, eval_datasplit_type)
elif 'synthetic_gaussian_scale' in config.data or 'enable_poisson_noise' in config.data:
    highres_data = get_data_without_synthetic_noise(data_dir, config, eval_datasplit_type)

if highres_data is not None:
    highres_data = ignore_pixels(highres_data).copy()


In [ ]:
print(f'{DataSplitType.name(eval_datasplit_type)}_P{custom_image_size}_G{image_size_for_grid_centers}_M{mmse_count}_Sk{ignored_last_pixels}')
print('PSNR on Highres', model.denoise_channel, avg_range_inv_psnr(highres_data[...,0], pred_unnorm[...,0]))
ssim_hres_mean, ssim_hres_std = avg_ssim(highres_data[...,0], pred_unnorm[...,0])
print('SSIM on Highres', model.denoise_channel, np.round(ssim_hres_mean,3), '±', np.round(ssim_hres_std,3))


In [ ]:
_,ax = plt.subplots(figsize=(12,8),ncols=3,nrows=2)
idx = np.random.randint(len(pred))
print(idx)
ax[0,0].imshow(tar_normalized[idx,...,0], cmap='magma')
ax[0,1].imshow(highres_data[idx,...,0], cmap='magma')
ax[0,2].imshow(pred_unnorm[idx,...,0], cmap='magma')
cropsz = 512
h_s = np.random.randint(0, tar_normalized.shape[1] - cropsz)
h_e = h_s + cropsz
w_s = np.random.randint(0, tar_normalized.shape[2] - cropsz)
w_e = w_s + cropsz

ax[1,0].imshow(tar_normalized[idx,h_s:h_e,w_s:w_e,0], cmap='magma')
ax[1,1].imshow(highres_data[idx,h_s:h_e,w_s:w_e,0], cmap='magma')
ax[1,2].imshow(pred_unnorm[idx,h_s:h_e,w_s:w_e,0], cmap='magma')
# Add rectangle to the region
rect = patches.Rectangle((w_s, h_s), w_e-w_s, h_e-h_s, linewidth=1, edgecolor='r', facecolor='none')
ax[0,0].add_patch(rect)


### To save to tiff file.

In [ ]:
def get_noise_str():
    noise_str = ''
    if 'synthetic_gaussian_scale' in config.data:
        noise_str  = f'_N{config.data.synthetic_gaussian_scale}'
    if 'poisson_noise_factor' in config.data and config.data.poisson_noise_factor is not None and config.data.poisson_noise_factor > 0:
        noise_str += f'_P{config.data.poisson_noise_factor}'
    
    return noise_str

def get_model_str():
    tokens = ckpt_dir.split('/')
    tokens.remove('')
    return '-'.join([x.replace('-','') for x in tokens[-3:]])

In [ ]:
from denoisplit.core.tiff_reader import save_tiff
from denoisplit.core.data_split_type import DataSplitType

assert pred_unnorm[...,0].std() > pred_unnorm[...,1].std()
denoised_pred = pred_unnorm[...,0].copy()
denoised_pred[denoised_pred<0] = 0
denoised_pred = denoised_pred.astype(np.uint16)
if denoise_channel == 'Ch1':
    fname = config.data.ch1_fname
elif denoise_channel == 'Ch2':
    fname = config.data.ch2_fname
elif denoise_channel == 'input':
    fname = 'input.tif'
fname = f'{DataSplitType.name(eval_datasplit_type)}Data_{get_model_str()}{get_noise_str()}_{fname}'
output_fpath = os.path.join(save_output_dir,fname)
print(output_fpath)
save_tiff(output_fpath, denoised_pred)


In [ ]:
!ls -lhrt "$output_fpath"

In [ ]:
d = load_tiff(output_fpath)
plt.imshow(d[0])